# Conception d'un DataMart sur l'ensemble des communes de Mayotte

## Contexte

L’objectif de ce projet est de construire un **DataMart** regroupant des données sur les communes de **Mayotte** afin d’analyser les conditions de logement, l’accès aux services essentiels, l’emploi, la précarité financière et la scolarisation.  

Il permettra de centraliser des informations clés telles que les types de logements, l’accès à l’eau et à l’électricité, la présence d’infrastructures, le taux de chômage, la pauvreté, ainsi que le niveau de formation des habitants.  

En s’appuyant sur des sources officielles, cet outil facilitera la prise de décision des acteurs locaux en fournissant des indicateurs précis sur la situation socio-économique et urbaine du territoire.

## Exploration des fichiers de données

En explorant les fichiers Excel, on peut déceler plusieurs feuilles/tables.

Pour le fichier "Data-Logement", on a les logements par village de résidence enrichi avec :  
- Les catégories de logement et leurs nombres
- Les aspects du bâti
- Les types de construction

Ensuite, pour le fichier "Data-résidence-accessibilité", on a les résidences principales par village enrichi avec :
- La nature du sol
- La taille des ménages
- L'accessibilité par voir carossable
- L'approvisionnement en eau
- La présence d'électricité dans logement
- L'évacuation des eaux usées
- La présence de bac à ordures
- La connexion à internet
- Le confort sanitaire de base

Et enfin, 4 fichiers Excel concernant par commune :
- La formation et la scolarisation
- La précarité dans les logements
- La précarité financière
- Le chômage et l'emploi

Pour pouvoir construire ce DataMart, on n'aura pas besoin de toutes ces feuilles. 
 
On va donc exclure :
- Les logements par village de résidence enrichi avec les aspects du bâti
- Les logements par village de résidence enrichi avec les types de construction
- Les résidences principales par village enrichi avec l'évacuation des eaux usées

Entre tous ces fichiers, on décèle deux clés/identifiants. Le code commune pour les communes et le code village associé au village de résidence.

## Préparation des fichiers de données

Pour gagner du temps lors de la jointure des fichiers sous Python, j'ai manipulé les fichiers Excel pour faire en sorte que lorsque je vais lire le fichier avec ses feuilles, je n'aurais pas besoin de spécifier les cellules que je veux.  
Donc j'ai dû retirer le titre des feuilles et les commentaires en bas des feuilles.

Pour la plupart des feuilles de données dans le fichier "Data-Logement", j'ai décidé de retirer les colonnes "Ensemble" et "Ensemble%" car c'est simplement le total des données pour chaque feuille.  
On peut donc facilement retrouver ces chiffres.

## Importation et jointures des fichiers sous Python

On a besoin de la librairie Pandas pour importer et manipuler les données.

In [ ]:
import pandas as pd

Vu qu'on a les données sur les communes et sur les villages, on va faire deux sous-datamarts pour ensuite en les fusionner et en avoir qu'un.

## Importation des données

In [54]:
# 1
dfLogement = pd.read_excel("data/Data-Logement.xls", "LOG1")
dfResidence_natSol = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC2")
dfResidence_menages = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC4")
dfResidence_carossable = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC5")
dfResidence_eau = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC8")
dfResidence_elec = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC10")
dfResidence_ordures = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC13")
dfResidence_internet = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC20")
dfResidence_sanitaire = pd.read_excel("data/Data-residence_accessibilite.xlsx", "PRINC7")

In [55]:
# 2
dfScolarisation = pd.read_excel("data/Table-Formation-scolarisation.xlsx")
dfPreLogement = pd.read_excel("data/Table_precarite_logement.xlsx")
dfPreFinanciere = pd.read_excel("data/Table_Precarite_Financiere.xlsx")
dfChomage = pd.read_excel("data/Table_emploi_chomage.xlsx")

## Création des deux sous-datamarts

In [45]:
dataMart1 = pd.merge(dfLogement, dfResidence_natSol, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = pd.merge(dataMart1, dfResidence_menages, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = pd.merge(dataMart1, dfResidence_carossable, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = pd.merge(dataMart1, dfResidence_eau, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = pd.merge(dataMart1, dfResidence_elec, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = pd.merge(dataMart1, dfResidence_ordures, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = pd.merge(dataMart1, dfResidence_internet, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = pd.merge(dataMart1, dfResidence_sanitaire, how="inner", on=["Code village", "Village de résidence"])
dataMart1 = dataMart1.astype({"Code village": str})
print(dataMart1)

   Code village Village de résidence  Résidence principale  \
0      97601001                Acoua                   927   
1      97601002          Mtsangadoua                   383   
2      97602001           Bandraboua                   664   
3      97602002             Handréma                   398   
4      97602003         Mtsangamboua                   298   
..          ...                  ...                   ...   
67     97616002             Mangajou                   387   
68     97617001             Tsingoni                   837   
69     97617002               Mroalé                   225   
70     97617003              Combani                  1582   
71     97617004             Miréréni                   806   

    Résidence principale%  Logement occasionnel  Logement occasionnel%  \
0               81.458699                     9               0.790861   
1               69.384058                     5               0.905797   
2               79.807692        

In [46]:
dataMart2 = pd.merge(dfScolarisation, dfPreLogement, how="inner", on="COMMUNE")
dataMart2 = pd.merge(dataMart2, dfPreFinanciere, how="inner", on="COMMUNE")
dataMart2 = pd.merge(dataMart2, dfChomage, how="inner", on="COMMUNE")
print(dataMart2)

                COMMUNE  \
0           Acoua (601)   
1      Bandraboua (602)   
2        Bandrele (603)   
3          Bouéni (604)   
4         Chiconi (605)   
5       Chirongui (606)   
6         Dembeni (607)   
7        Dzaoudzi (608)   
8       Kani-Kéli (609)   
9         Koungou (610)   
10      Mamoudzou (611)   
11      Mtsamboro (612)   
12  M'tsangamouji (613)   
13       Ouangani (614)   
14       Pamandzi (615)   
15           Sada (616)   
16       Tsingoni (617)   
17              MAYOTTE   

    Nombre d'enfants bénéficiaires de l'Allocation de Rentrée Scolaire (ARS) au 31 décembre 2017   \
0                                                 883                                               
1                                                1680                                               
2                                                1160                                               
3                                                 980                             

## Fusion des deux sous-datamarts

Pour pouvoir les fusionner, on doit créer une clé commune.

Pour cela, on peut créer une colonne "code commune" dans les deux sous-datamarts avec la colonne "code village" et la colonne "COMMUNE".

In [47]:
dataMart1.insert(1, "Code commune", dataMart1["Code village"].str.slice(2,5))
dataMart2.insert(1, "Code commune", dataMart2["COMMUNE"].str.slice(start=-4, stop=-1))

In [52]:
datamart = pd.merge(dataMart2, dataMart1, how="inner", on="Code commune")
datamart.insert(datamart.columns.get_loc("Code commune") + 1, "Village de résidence", datamart.pop("Village de résidence"))
datamart.insert(datamart.columns.get_loc("Village de résidence") + 1, "Code village", datamart.pop("Code village"))
print(datamart)

             COMMUNE Code commune Village de résidence Code village  \
0        Acoua (601)          601                Acoua     97601001   
1        Acoua (601)          601          Mtsangadoua     97601002   
2   Bandraboua (602)          602           Bandraboua     97602001   
3   Bandraboua (602)          602             Handréma     97602002   
4   Bandraboua (602)          602         Mtsangamboua     97602003   
..               ...          ...                  ...          ...   
67        Sada (616)          616             Mangajou     97616002   
68    Tsingoni (617)          617             Tsingoni     97617001   
69    Tsingoni (617)          617               Mroalé     97617002   
70    Tsingoni (617)          617              Combani     97617003   
71    Tsingoni (617)          617             Miréréni     97617004   

    Nombre d'enfants bénéficiaires de l'Allocation de Rentrée Scolaire (ARS) au 31 décembre 2017   \
0                                             

## Export du DataMart sous Excel

In [58]:
datamart.to_excel("DataMart.xlsx", index = False)